##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = "W3sicGFydHMiOiBbeyJ0ZXh0IjogIldoYXQncyBpbiB0aGlzIHBpY3R1cmU/In0sIHsiZmlsZV9kYXRhIjogeyJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2dlbmVyYXRpdmVhaS1kb3dubG9hZHMvaW1hZ2VzL3Njb25lcy5qcGciLCAibWltZV90eXBlIjogImltYWdlL2pwZWcifX1dfV0="  # @param {isTemplate: true}
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

Uploading: https://storage.googleapis.com/generativeai-downloads/images/scones.jpg
[
    {
        "parts": [
            {
                "text": "What's in this picture?"
            },
            {
                "file_data": {
                    "mime_type": "image/jpeg",
                    "file_uri": "https://generativelanguage.googleapis.com/v1beta/files/ypcaj8r9euho"
                }
            }
        ]
    }
]


## Call `generate_content`

In [3]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

That's a lovely overhead shot of a rustic-style breakfast or brunch spread.  Here's what's in the picture:

* **Blueberry scones:** Several freshly baked blueberry scones are arranged on a piece of parchment paper.  They appear to be crumbly and slightly sweet.
* **Fresh blueberries:** A small bowl of fresh blueberries sits alongside the scones.  There are also some loose blueberries scattered on the paper.
* **Coffee:** Two small cups of coffee are present; one seems to be almost full, and the other about half full.
* **Peonies:** Several pink peonies are arranged next to the food, adding a touch of elegance and color.
* **Spoon:** A decorative spoon with "Let's Jam" engraved on it.
* **Parchment paper:** The scones and blueberries are presented on a piece of parchment paper with some purple stains.
* **Dark background:** The entire scene is set against a dark, textured surface.

The overall impression is one of a relaxed and aesthetically pleasing meal, perfect for a leisurely morning.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [4]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

What's in this picture? 




-------------------------------------------------------------------------------- 



In [7]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("/content/file1.jpg", mime_type=None),
  upload_to_gemini("/content/file2.jpg", mime_type=None),
  upload_to_gemini("/content/file2.jpg", mime_type=None),
]

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
        files[1],
        files[2],
        "Identify the shoes name and brand, and the player name ",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Certainly! Here's the information for each image:\n\n**Image 1:**\n\n*   **Player:** Kevin Durant\n*   **Shoe Brand:** Nike\n*   **Shoe Name:** Nike KD 6\n\n**Image 2 and 3:**\n\n*   **Player:** Stephen Curry\n*   **Shoe Brand:** Under Armour\n*   **Shoe Name:** Under Armour Curry 1",
      ],
    },
    {
      "role": "user",
      "parts": [
        "For each player and shoe identified, add as much data as possible ",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, let's dive into more detail about each player and their respective shoes!\n\n**Image 1: Kevin Durant and the Nike KD 6**\n\n*   **Player:** Kevin Durant\n    *   **Full Name:** Kevin Wayne Durant\n    *   **Nickname:** KD, The Slim Reaper\n    *   **Position:** Forward\n    *   **NBA Teams:** He has played for the Seattle SuperSonics/Oklahoma City Thunder, Golden State Warriors, Brooklyn Nets, and Phoenix Suns.\n    *   **Accolades:** \n        *   2× NBA champion (2017, 2018)\n        *   2× NBA Finals MVP (2017, 2018)\n        *   NBA Most Valuable Player (2014)\n        *   4× NBA scoring champion (2010–2012, 2014)\n        *   13× NBA All-Star (2010–2019, 2021-2023)\n        *   6x All-NBA first team (2010-2014, 2018)\n        *   2008 NBA Rookie of the Year\n        *   Olympic gold medalist (2012, 2016, 2020)\n        *   Basketball World Cup gold medalist (2010)\n    *   **Playing Style:** Known for his exceptional scoring ability, smooth jump shot, and versatility.\n\n*   **Shoe:** Nike KD 6\n    *   **Brand:** Nike\n    *   **Release Year:** 2013-2014\n    *   **Designer:** Leo Chang\n    *   **Key Features:**\n        *   Low-cut design, which was a departure from previous KD models.\n        *   Flywire technology for a locked-down fit.\n        *   Hyperfuse construction for breathability and durability.\n        *   Zoom Air cushioning in the forefoot for responsiveness.\n        *   Unique design elements and colorways inspired by Durant's personal interests and stories.\n        *   The \"OKC\" colorway was especially popular, featuring the Thunder's team colors.\n        *   Multiple popular colorways were released, such as the \"Maryland Blue Crab\" and the \"Aunt Pearl\".\n    *   **Significance:** The KD 6 was a significant model in Durant's signature line, celebrated for its performance, sleek design, and storytelling aspects. It was a very popular shoe during his MVP season with the Thunder.\n\n**Image 2 & 3: Stephen Curry and the Under Armour Curry 1**\n\n*   **Player:** Stephen Curry\n    *   **Full Name:** Wardell Stephen Curry II\n    *   **Nickname:** Steph, Chef Curry\n    *   **Position:** Point Guard\n    *   **NBA Team:** Golden State Warriors\n    *   **Accolades:**\n        *   4× NBA champion (2015, 2017, 2018, 2022)\n        *   NBA Finals MVP (2022)\n        *   2× NBA Most Valuable Player (2015, 2016)\n        *   NBA All-Star Game MVP (2022)\n        *   NBA All-Star (9x)\n        *   9x All-NBA selection\n        *   NBA scoring champion (2016, 2021)\n        *   NBA steals champion (2016)\n        *   He is considered to be the best shooter of all time.\n    *   **Playing Style:** Known for his exceptional three-point shooting range, ball-handling, and off-ball movement. He revolutionized the NBA with his three point ability.\n\n*   **Shoe:** Under Armour Curry 1\n    *   **Brand:** Under Armour\n    *   **Release Year:** 2015\n    *   **Designer:** Dave Dombrow, Ryan Drew, and Rob Walker\n    *   **Key Features:**\n        *   The first signature shoe for Stephen Curry with Under Armour, making it a landmark moment for the brand.\n        *   Anafoam upper for support and structure.\n        *   Charged Cushioning midsole for energy absorption and responsiveness.\n        *   Outsole herringbone pattern for grip.\n        *   The shoe featured a clean, streamlined design.\n        *   Many colorways were released to represent the Golden State Warriors and Curry's personality.\n        *   The \"Dub Nation\" colorway and \"MVP\" colorways were popular.\n    *   **Significance:** The Curry 1 marked the beginning of Curry's signature shoe line and played a significant role in Under Armour's growth in the basketball market. The popularity of the Curry 1 was directly tied to Curry's MVP season and championship run, solidifying it as an iconic basketball sneaker.\n\nLet me know if you have any other questions or would like more information about anything!\n",
      ],
    },
  ]
)

response = chat_session.send_message("INSERT_INPUT_HERE")

print(response.text)

Uploaded file 'file1.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/thd0ppbm77j4
Uploaded file 'file2.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/cof3qsosgzn7
Uploaded file 'file2.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/zgfwutx18dq7
Please provide the input you would like me to use. I'm ready for your next request!



In [11]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("/content/file1.jpg", mime_type=None),
  upload_to_gemini("/content/file2.jpg", mime_type=None),
]

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
        files[1],
      ],
    }
  ]
)

response = chat_session.send_message("Identify the shoes name and brand, and the player name. For each player and shoe identified, add as much data as possible. At the end of the results Return results as a table with Shoe Brand, Name, Colorway, Player, team, important metadata. Also, for each shoe, givbe the best description to search for exact match over google in order to purchase one")

print(response.text)

Uploaded file 'file1.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/vam2kdvr162y
Uploaded file 'file2.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/vc7473epcgu
Certainly! Here's the breakdown of the players, shoes, and their details from the images, followed by a structured table summarizing the information.

**Image 1 Analysis**

*   **Player:** Kevin Durant
*   **Shoes:** Nike KD 7 
*   **Description for search:** "Nike KD 7 basketball shoes" + the specific colorway. 

**Image 2 Analysis**

*   **Player:** Stephen Curry
*   **Shoes:** Under Armour Curry 1
*  **Description for search:** "Under Armour Curry 1 basketball shoes" + the specific colorway.

**Table of Results**

| Shoe Brand   | Shoe Name     | Colorway                                | Player         | Team                      | Important Metadata                                   | Best Search Description                                                  |
|--------------|---------------|

In [13]:
import base64
from openai import OpenAI

client = OpenAI(
    api_key="",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
base64_image = encode_image("/content/file1.jpg")

response = client.chat.completions.create(
  model="gemini-1.5-flash",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Identify the shoes name and brand, and the player name. For each player and shoe identified, add as much data as possible. At the end of the results Return results as a table with Shoe Brand, Name, Colorway, Player, team, important metadata. Also, for each shoe, givbe the best description to search for exact match over google in order to purchase one",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)

print(response.choices[0].content)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here\'s the information you requested, presented in table format.  Note that determining the exact colorway from an image can be subjective, so the colorway description might vary slightly depending on interpretation.\n\n| Shoe Brand | Name | Colorway | Player | Team | Important Metadata | Google Search Query |\n|---|---|---|---|---|---|---|\n| Nike | KD 9 | Navy/White/Gold | Kevin Durant | Oklahoma City Thunder | Released in 2016; signature shoe for Kevin Durant; features Zoom Air cushioning; flyknit upper | "Nike KD 9 Navy White Gold" |\n| Nike | Kobe AD | Purple/Gold | Kobe Bryant | Los Angeles Lakers | Released in 2017; signature shoe for Kobe Bryant, features Lunarlon cushioning and adaptive traction | "Nike Kobe AD Purple Gold" |\n\n\n**Additional Data:**\n\n**Kevin Durant:**\n\n* **Full Name:** Kevin Ray Durant\n* **Position:**  Small Forward/Power Forward\n* **NBA Draft:** 2007, 2nd over

In [16]:
print(response.choices[0].message.content)

Here's the information you requested, presented in table format.  Note that determining the exact colorway from an image can be subjective, so the colorway description might vary slightly depending on interpretation.

| Shoe Brand | Name | Colorway | Player | Team | Important Metadata | Google Search Query |
|---|---|---|---|---|---|---|
| Nike | KD 9 | Navy/White/Gold | Kevin Durant | Oklahoma City Thunder | Released in 2016; signature shoe for Kevin Durant; features Zoom Air cushioning; flyknit upper | "Nike KD 9 Navy White Gold" |
| Nike | Kobe AD | Purple/Gold | Kobe Bryant | Los Angeles Lakers | Released in 2017; signature shoe for Kobe Bryant, features Lunarlon cushioning and adaptive traction | "Nike Kobe AD Purple Gold" |


**Additional Data:**

**Kevin Durant:**

* **Full Name:** Kevin Ray Durant
* **Position:**  Small Forward/Power Forward
* **NBA Draft:** 2007, 2nd overall pick by Seattle SuperSonics
* **Championships:** 2 (2017, 2023)
* **MVP:** 2014
* **Finals MVP:** 2017,

In [17]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("/content/file3.jpg", mime_type=None),
  # upload_to_gemini("/content/file2.jpg", mime_type=None),
]

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    }
  ]
)

response = chat_session.send_message("Identify the hand watch name and brand, and the person name if possible. For each watch and person identified, add as much data as possible. At the end of the results Return results as a table with watch Brand, Name, Colorway, person, important metadata. Also, for each watch, givbe the best description to search for exact match over google in order to purchase one")

print(response.text)

Uploaded file 'file3.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/43tml762woyl
Certainly! Here's a breakdown of the items and people in the image, along with a table summarizing the information.

**Identification**

**Person**

*   **Left:** The person on the left is **Rosie Huntington-Whiteley.** 

    *   **Metadata:** Rosie Huntington-Whiteley is a British model and actress. She is known for her work with Victoria's Secret and appearances in films such as "Transformers: Dark of the Moon" and "Mad Max: Fury Road." 
    *   She is also a prominent fashion influencer and businesswoman.

*   **Right:** The person on the right is **Jason Statham.**

    *   **Metadata:** Jason Statham is an English actor, producer, and martial artist. He's known for his roles in action films like "The Transporter," "Crank," "The Expendables," and the "Fast & Furious" franchise. He is well known for his baldness and iconic tough-guy image.

**Handwatch**

*   **Brand:** The handwatch wo

In [18]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("/content/file3.jpg", mime_type=None),
  # upload_to_gemini("/content/file2.jpg", mime_type=None),
]

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    }
  ]
)

response = chat_session.send_message("Identify the hand watch name and brand, and the person name if possible. For each watch and person identified, add as much data as possible. At the end of the results Return results as a table with watch Brand, Name, Colorway, approximate price, person, important metadata. Also, for each watch, givbe the best description to search for exact match over google in order to purchase one")

print(response.text)

Uploaded file 'file3.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/8yzqkta1lc2y
Okay, let's analyze the image and identify the watch and the people.

**People Identification:**

*   **Woman:** The woman is **Rosie Huntington-Whiteley**. She's a well-known English model, actress, and businesswoman. She is known for her work with Victoria's Secret and is a successful model and entrepreneur.

*   **Man:** The man is **Jason Statham**. He is an English actor known for his roles in action films. He is also a former diver.

**Watch Identification:**

The watch on Jason Statham's wrist is a **Patek Philippe Nautilus**. 

**Watch Details:**

*   **Brand:** Patek Philippe
*   **Model Name:** Nautilus
*   **Colorway:** It appears to be the **rose gold** version based on its color and the reference
*   **Approximate Price:** The retail price on this watch has a price tag close to **$60,000 usd to $70,000** depending of the market
*   **Description to search for purchase:** "Pate